In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
# %load_ext cudf.pandas
# import pandas as pd
# print(pd)


In [ ]:
# Optimizations
# GDAL optimizations
import multiprocessing as mp
import os

cpu_count: int = mp.cpu_count()
num_cores: int = cpu_count - 2
os.environ["GDAL_NUM_THREADS"] = f"{num_cores}"
os.environ["GDAL_CACHEMAX"] = "1024"


## Libraries

In [ ]:
# Imports
from pathlib import Path
import numpy as np


In [ ]:
# Add root to path
import sys

sys.path.append("..")
from component.script.utilities.file_filter import (
    list_files_by_extension,
)


## Set user parameters

In [ ]:
project_name = "test"


In [ ]:
forest_source = "gfc"  ##gfc, tmf
tree_cover_threshold = 10
years = [2015, 2020, 2024]
string_years = [str(num) for num in years]


In [ ]:
n_samples = 10000
random_seed = 1
spatial_cell_size_km = 10
adapt = True


## Connect folders

In [ ]:
root_folder: Path = Path.cwd().parent
downloads_folder: Path = root_folder / "data"
downloads_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
project_folder = downloads_folder / project_name
project_folder.mkdir(parents=True, exist_ok=True)
processed_data_folder = project_folder / "data"
processed_data_folder.mkdir(parents=True, exist_ok=True)
sampling_folder = project_folder / "far_samples"
sampling_folder.mkdir(parents=True, exist_ok=True)

## Helper functions

In [ ]:
def filter_files(input_files, filter_words, exclude_words=None, include_all=True):
    """
    Filters a list of files based on include and exclude words.
    Parameters:
        input_files (list): List of file paths to be filtered.
        filter_words (list): Words that must be present in the filenames for inclusion.
        exclude_words (list, optional): Words that must not be present in the filenames for exclusion. Defaults to None.
        include_all (bool, optional): If True, all filter words must be present in the filename. If False, at least one of the filter words must be present. Defaults to False.
    Returns:
        list: Filtered list of files.
    """
    # Ensure all words are lowercase for case-insensitive comparison
    filter_words = [word.lower() for word in filter_words]
    exclude_words = [word.lower() for word in (exclude_words or [])]

    if include_all:
        filtered_files = [
            file
            for file in input_files
            if all(word in Path(file).name.lower() for word in filter_words)
            and not any(
                exclude_word in Path(file).name.lower()
                for exclude_word in exclude_words
            )
        ]
    else:
        filtered_files = [
            file
            for file in input_files
            if any(word in Path(file).name.lower() for word in filter_words)
            and not any(
                exclude_word in Path(file).name.lower()
                for exclude_word in exclude_words
            )
        ]
    
    return filtered_files

## Select forest cover change file

In [ ]:
# List all raster files in the processed data folder
input_raster_files = list_files_by_extension(processed_data_folder, [".tiff", ".tif"])
input_raster_files


In [ ]:
forest_change_file = filter_files(
    input_raster_files,
    ["forest", "loss", forest_source] + [str(num) for num in years],
    ["distance", "edge"],
)[0]
forest_change_file

## Periods dictionaries

In [ ]:
calibration_dict = {
    "period": "calibration",
    "train_period": "calibration",
    "initial_year": years[0],
    "final_year": years[1],
    "defor_value": 1,
    "time_interval": years[1] - years[0],
}
validation_dict = {
    "period": "validation",
    "train_period": "calibration",
    "initial_year": years[1],
    "final_year": years[2],
    "defor_value": 1,
    "time_interval": years[2] - years[1],
}
historical_dict = {
    "period": "historical",
    "train_period": "historical",
    "initial_year": years[0],
    "final_year": years[2],
    "defor_value": [1, 2],
    "time_interval": years[2] - years[0],
}
forecast_dict = {
    "period": "forecast",
    "train_period": "historical",
    "initial_year": years[0],
    "final_year": years[2],
    "defor_value": [1, 2],
    "time_interval": years[2] - years[0],
}


In [ ]:
# Crear el diccionario principal
period_dict = {
    calibration_dict["period"]: calibration_dict,
    validation_dict["period"]: validation_dict,
    historical_dict["period"]: historical_dict,
    forecast_dict["period"]: forecast_dict,
}


## Select input files based on period

In [ ]:
def get_variable_independant_files(input_raster_files, period):
    # Define the period-independent variables and their associated files
    period_independant_variables = ["altitude", "slope", "pa", "subj"]
    altitude_files = filter_files(input_raster_files, ["altitude"], None, False)
    slope_files = filter_files(input_raster_files, ["slope"], None, False)
    wdpa_files = filter_files(input_raster_files, ["pa"], None, False)
    subj_files = filter_files(input_raster_files, ["subj"], None, False)

    # Define the rivers and roads variables and their associated files
    rivers_files = filter_files(
        input_raster_files, ["rivers", "reprojected", "distance"], None, True
    )
    road_files = filter_files(
        input_raster_files, ["roads", "reprojected", "distance"], None, True
    )

    # Define the period-dependent variables and their associated files
    period_dictionary = period_dict[period]
    initial_year = str(period_dictionary["initial_year"])
    final_year = str(period_dictionary["final_year"])
    exclude_year = ", ".join(
        map(
            str,
            set(years)
            - {period_dictionary["initial_year"], period_dictionary["final_year"]},
        )
    )
    forest_loss_files = filter_files(
        input_raster_files,
        [forest_source, initial_year, final_year, "forest", "loss"],
        [exclude_year, "edge"],
        True,
    )
    forest_edge_files = filter_files(
        input_raster_files,
        [forest_source, initial_year, "forest", "reprojected", "edge"],
        None,
        True,
    )
    town_files = filter_files(
        input_raster_files,
        [initial_year, "town", "reprojected", "distance"],
        None,
        True,
    )

    # Create a dictionary with variable types as keys and file paths as values
    variable_file_mapping = {
        "period": period_dictionary["period"],
        "altitude": altitude_files[0],
        "slope": slope_files[0],
        "pa": wdpa_files[0],
        "subj": subj_files[0],
        "dist_river": rivers_files[0],
        "dist_road": road_files[0],
        "dist_town": town_files[0],
        "fcc": forest_loss_files[0],
        "dist_edge": forest_edge_files[0],
    }
    return variable_file_mapping


In [ ]:
calibration_files = get_variable_independant_files(input_raster_files, "calibration")
validation_files = get_variable_independant_files(input_raster_files, "validation")
historical_files = get_variable_independant_files(input_raster_files, "historical")
forecast_files = get_variable_independant_files(input_raster_files, "forecast")

## Generate sample for the period

In [ ]:
"""Sample points and extract raster values."""

# Import
from glob import glob  # To explore files in a folder
import os  # Operating system interfaces
import sys  # To read and write files
import uuid

# Third party imports
import numpy as np  # For arrays
from osgeo import gdal  # GIS libraries
import pandas as pd  # To export result as a pandas DF

# Local imports
from forestatrisk.misc import makeblock, progress_bar


def sample(
    nsamp=10000,
    adapt=True,
    seed=1234,
    csize=10,
    var_dictionary="dictionary_of_files.txt",
    blk_rows=0,
    verbose=True,
):
    """Sample points and extract raster values.

    This function (i) randomly draws spatial points in deforested and
    forested areas and (ii) extract environmental variable values for
    each spatial point.

    :param nsamp: Number of random spatial points.

    :param adapt: Boolean. Adapt ``nsamp`` to forest area: 1000 for 1 Mha of
        forest, with min=10000 and max=50000. Default to ``True``.

    :param seed: Seed for random number generator.

    :param csize: Spatial cell size in km.

    :param var_dir: Directory with raster data.

    :param blk_rows: If > 0, number of lines per block.

    :param verbose: Toogle progress bar.

    :return: A Pandas DataFrame, each row being one observation.

    """

    period_output_folder = sampling_folder / var_dictionary["period"]
    if not os.path.exists(period_output_folder):
        os.makedirs(period_output_folder)

    # Set random seed
    np.random.seed(seed)

    # =============================================
    # Sampling pixels
    # =============================================

    if verbose:
        text = "Sample 2x {} pixels (deforested vs. forest)"
        print(text.format(nsamp))

    # Read defor raster
    forest_raster_file = var_dictionary["fcc"]
    forestR = gdal.Open(forest_raster_file)
    forestB = forestR.GetRasterBand(1)

    # Make blocks
    blockinfo = makeblock(forest_raster_file, blk_rows=blk_rows)
    nblock = blockinfo[0]
    nblock_x = blockinfo[1]
    x = blockinfo[3]
    y = blockinfo[4]
    nx = blockinfo[5]
    ny = blockinfo[6]
    if verbose:
        text = "Divide region in {} blocks"
        print(text.format(nblock))

    # Number of defor/forest pixels by block
    if verbose:
        text = "Compute number of deforested and forest pixels per block"
        print(text)
    ndc = 0
    ndc_block = np.zeros(nblock, dtype=int)
    nfc = 0
    nfc_block = np.zeros(nblock, dtype=int)

    # Loop on blocks of data
    for b in range(nblock):
        # Progress bar
        if verbose:
            progress_bar(nblock, b + 1)
        # Position in 1D-arrays
        px = b % nblock_x
        py = b // nblock_x
        # Read the data
        forest = forestB.ReadAsArray(x[px], y[py], nx[px], ny[py])
        # Identify pixels (x/y coordinates) which are deforested
        deforpix = np.nonzero(forest == 0)
        ndc_block[b] = len(deforpix[0])  # Number of defor pixels
        ndc += len(deforpix[0])
        # Identify pixels (x/y coordinates) which are forest
        forpix = np.nonzero(forest == 1)
        nfc_block[b] = len(forpix[0])  # Number of forest pixels
        nfc += len(forpix[0])

    # Adapt nsamp to forest area
    if adapt is True:
        gt = forestR.GetGeoTransform()
        pix_area = gt[1] * (-gt[5])
        farea = pix_area * (nfc + ndc) / 10000  # farea in ha
        nsamp_prop = 1000 * farea / 1e6  # 1000 per 1Mha
        if nsamp_prop >= 50000:
            nsamp = 50000
        elif nsamp_prop <= 10000:
            nsamp = 10000
        else:
            nsamp = int(np.rint(nsamp_prop))

    # Proba of drawing a block
    if verbose:
        print("Draw blocks at random")
    proba_block_d = ndc_block / ndc
    proba_block_f = nfc_block / nfc
    # Draw block number nsamp times
    block_draw_d = np.random.choice(
        list(range(nblock)), size=nsamp, replace=True, p=proba_block_d
    )
    block_draw_f = np.random.choice(
        list(range(nblock)), size=nsamp, replace=True, p=proba_block_f
    )
    # Number of times the block is drawn
    nblock_draw_d = np.zeros(nblock, dtype=int)
    nblock_draw_f = np.zeros(nblock, dtype=int)
    for s in range(nsamp):
        nblock_draw_d[block_draw_d[s]] += 1
        nblock_draw_f[block_draw_f[s]] += 1

    # Draw defor/forest pixels in blocks
    if verbose:
        print("Draw pixels at random in blocks")
    # Object to store coordinates of selected pixels
    deforselect = np.empty(shape=(0, 2), dtype=int)
    forselect = np.empty(shape=(0, 2), dtype=int)
    # Loop on blocks of data
    for b in range(nblock):
        # Progress bar
        if verbose:
            progress_bar(nblock, b + 1)
        # nbdraw
        nbdraw_d = nblock_draw_d[b]
        nbdraw_f = nblock_draw_f[b]
        # Position in 1D-arrays
        px = b % nblock_x
        py = b // nblock_x
        # Read the data
        forest = forestB.ReadAsArray(x[px], y[py], nx[px], ny[py])
        # Identify pixels (x/y coordinates) which are deforested
        # !! Values returned in row-major, C-style order (y/x) !!
        deforpix = np.nonzero(forest == 0)
        deforpix = np.transpose((x[px] + deforpix[1], y[py] + deforpix[0]))
        ndc_block = len(deforpix)
        # Identify pixels (x/y coordinates) which are forested
        forpix = np.nonzero(forest == 1)
        forpix = np.transpose((x[px] + forpix[1], y[py] + forpix[0]))
        nfc_block = len(forpix)
        # Sample deforested pixels
        if nbdraw_d > 0:
            if nbdraw_d < ndc_block:
                i = np.random.choice(ndc_block, size=nbdraw_d, replace=False)
                deforselect = np.concatenate((deforselect, deforpix[i]), axis=0)
            else:
                # nbdraw = ndc_block
                deforselect = np.concatenate((deforselect, deforpix), axis=0)
        # Sample forest pixels
        if nbdraw_f > 0:
            if nbdraw_f < nfc_block:
                i = np.random.choice(nfc_block, size=nbdraw_f, replace=False)
                forselect = np.concatenate((forselect, forpix[i]), axis=0)
            else:
                # nbdraw = ndc_block
                forselect = np.concatenate((forselect, forpix), axis=0)

    # =============================================
    # Compute center of pixel coordinates
    # =============================================
    if verbose:
        print("Compute center of pixel coordinates")

    # Landscape variables from forest raster
    gt = forestR.GetGeoTransform()
    ncol_r = forestR.RasterXSize
    nrow_r = forestR.RasterYSize
    Xmin = gt[0]
    Xmax = gt[0] + gt[1] * ncol_r
    Ymin = gt[3] + gt[5] * nrow_r
    Ymax = gt[3]

    # Dereference driver
    forestB = None
    del forestR

    # Concatenate selected pixels
    select = np.concatenate((deforselect, forselect), axis=0)

    # Offsets and coordinates #
    xOffset = select[:, 0]
    yOffset = select[:, 1]
    pts_x = (xOffset + 0.5) * gt[1] + gt[0]  # +0.5 for center of pixels
    pts_y = (yOffset + 0.5) * gt[5] + gt[3]  # +0.5 for center of pixels

    # ================================================
    # Compute cell number for spatial autocorrelation
    # ================================================

    # Cell number from region
    if verbose:
        text = "Compute number of {} x {} km spatial cells"
        print(text.format(csize, csize))
    csize = csize * 1000  # Transform km in m
    ncol = int(np.ceil((Xmax - Xmin) / csize))
    nrow = int(np.ceil((Ymax - Ymin) / csize))
    ncell = ncol * nrow
    if verbose:
        text = "... {} cells ({} x {})"
        print(text.format(ncell, nrow, ncol))
    # bigI and bigJ are the coordinates of the cells and start at zero
    if verbose:
        print("Identify cell number from XY coordinates")
    bigJ = ((pts_x - Xmin) / csize).astype(int)
    bigI = ((Ymax - pts_y) / csize).astype(int)
    cell = bigI * ncol + bigJ  # Cell number starts at zero

    # =============================================
    # Extract values from rasters
    # =============================================

    # Raster list
    # Extract keys excluding 'fcc' and sort them
    sorted_keys = sorted([key for key in var_dictionary.keys() if key != "period"])

    # Retrieve the corresponding file paths based on the sorted keys
    raster_list = [var_dictionary[key] for key in sorted_keys]

    # Make vrt with gdal.BuildVRT
    # Note: Extent and resolution from forest raster!
    if verbose:
        text = "Make virtual raster with variables as raster bands"
        print(text)
    param = gdal.BuildVRTOptions(
        resolution="user",
        outputBounds=(Xmin, Ymin, Xmax, Ymax),
        xRes=gt[1],
        yRes=-gt[5],
        separate=True,
    )
    rand_uuid = uuid.uuid4()
    vrt_file = f"/vsimem/var_{rand_uuid}.vrt"
    gdal.BuildVRT(vrt_file, raster_list, options=param)
    stack = gdal.Open(vrt_file)

    # List of nodata values
    nband = stack.RasterCount
    bandND = np.zeros(nband)
    for k in range(nband):
        band = stack.GetRasterBand(k + 1)
        bandND[k] = band.GetNoDataValue()
        if bandND[k] is None:
            print(
                "NoData value is not specified \
            for input raster file "
                + raster_list[k]
            )
            sys.exit(1)

    # Numpy array to store values
    nobs = select.shape[0]
    val = np.zeros(shape=(nobs, nband), dtype=float)

    # Extract raster values
    if verbose:
        text = "Extract raster values for selected pixels"
        print(text)
    for i in range(nobs):
        # Progress bar
        if verbose:
            progress_bar(nobs, i + 1)
        # ReadArray for extract
        extract = stack.ReadAsArray(int(xOffset[i]), int(yOffset[i]), 1, 1)
        val[i, :] = extract.reshape(
            nband,
        )

    # Close stack
    del stack

    # Replace NA
    # NB: ReadAsArray return float32 type
    bandND = bandND.astype(np.float32)
    for k in range(nband):
        val[val[:, k] == bandND[k], k] = np.nan

    # Add XY coordinates and cell number
    pts_x.shape = (nobs, 1)
    pts_y.shape = (nobs, 1)
    cell.shape = (nobs, 1)
    val = np.concatenate((val, pts_x, pts_y, cell), axis=1)

    # =============================================
    # Export and return value
    # =============================================

    # Save csize for interpolation of rhos
    ofile = os.path.join(period_output_folder, "csize_icar.txt")
    with open(ofile, "w", encoding="utf-8") as f:
        csize_km = csize / 1000
        f.write(str(csize_km))
    output_file = str(period_output_folder / "sample.txt")
    if verbose:
        text = "Export results to file {}"
        print(text.format(output_file))

    # Write to file by row
    colname = sorted_keys
    # for (i, j) in enumerate(sorted_keys):
    #     base_name = os.path.basename(j)
    #     index_dot = base_name.index(".")
    #     colname[i] = base_name[:index_dot]

    varname = ",".join(colname) + ",X,Y,cell"
    np.savetxt(output_file, val, header=varname, fmt="%s", delimiter=",", comments="")

    # Convert to pandas DataFrame and return the result
    colname.extend(["X", "Y", "cell"])
    val_df = pd.DataFrame(val, columns=colname)

    # Remove NA from data-set (otherwise scale() and
    # model_binomial_iCAR don't work)
    dataset = val_df.dropna(axis=0).copy()
    print(f"Number of samples: {len(dataset)}")
    # Set number of trials to one for far.model_binomial_iCAR()
    dataset.loc[:, "trial"] = 1  # Sample size
    ndefor = sum(dataset.fcc == 0)
    nfor = sum(dataset.fcc == 1)
    ifile = str(period_output_folder / "sample_size.csv")
    with open(ifile, "w", encoding="utf-8") as file:
        file.write("Var, n\n")
        file.write(f"ndefor, {ndefor}\n")
        file.write(f"nfor, {nfor}\n")
    print(f"Sample size: ndefor = {ndefor}, nfor = {nfor}")


# End


In [ ]:
calibration_samples = sample(
    n_samples, adapt, random_seed, spatial_cell_size_km, calibration_files
)


In [ ]:
historical_samples = sample(
    n_samples, adapt, random_seed, spatial_cell_size_km, historical_files
)
